In [1]:
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.preprocessing import minmax_scale
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score, plot_confusion_matrix
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.feature_extraction import DictVectorizer
from sklearn import preprocessing
from sklearn import ensemble
import importlib
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.utils import resample
from sklearn import svm
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from xgboost import XGBRegressor
import xgboost as xgb
from xgboost import plot_tree

from graphviz import Digraph


from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

In [2]:
data = pd.read_csv(r'uk_obs_coordinates_000000000000.csv')

In [3]:
data

,Site,Level,ClientMacAddr,lat,lng,localtime
0,UK Office,1st Floor,9c:8c:6e:46:1c:5e,51.460445,-0.933048,2020-09-01 23:58:49.582 UTC
1,UK Office,1st Floor,9c:8c:6e:46:0b:7c,51.460788,-0.932288,2020-09-01 23:58:45.435 UTC
2,UK Office,1st Floor,9c:8c:6e:46:1c:5e,51.460445,-0.933048,2020-09-01 23:58:38.112 UTC
3,UK Office,1st Floor,9c:8c:6e:46:0b:7c,51.460782,-0.932292,2020-09-01 23:58:35.379 UTC
4,UK Office,1st Floor,9c:8c:6e:46:0b:7c,51.460782,-0.932292,2020-09-01 23:58:29.827 UTC
...,...,...,...,...,...,...
3733370,UK Office,1st Floor,c4:6e:1f:1a:68:a1,51.460701,-0.932252,2020-03-21 23:26:33.958 UTC
3733371,UK Office,1st Floor,a4:e9:75:77:7e:c3,51.460475,-0.932350,2020-03-21 23:26:28.204 UTC
3733372,UK Office,1st Floor,9c:8c:6e:46:0b:7c,51.460774,-0.932317,2020-03-21 23:26:27.926 UTC
3733373,UK Office,1st Floor,c4:6e:1f:1a:68:a1,51.460701,-0.932252,2020-03-21 23:26:27.853 UTC


In [4]:
data = data.dropna()

In [5]:
data['localtime'] = data['localtime'].str.rstrip(' UTC')

In [6]:
d = [['1', 'Mobile']]
MobileOrFixed = pd.DataFrame(d, columns=['ClientMacAddr', 'Status'])
MobileOrFixed

,ClientMacAddr,Status
0,1,Mobile


In [7]:
datat = data[:]
datat

,Site,Level,ClientMacAddr,lat,lng,localtime
0,UK Office,1st Floor,9c:8c:6e:46:1c:5e,51.460445,-0.933048,2020-09-01 23:58:49.582
1,UK Office,1st Floor,9c:8c:6e:46:0b:7c,51.460788,-0.932288,2020-09-01 23:58:45.435
2,UK Office,1st Floor,9c:8c:6e:46:1c:5e,51.460445,-0.933048,2020-09-01 23:58:38.112
3,UK Office,1st Floor,9c:8c:6e:46:0b:7c,51.460782,-0.932292,2020-09-01 23:58:35.379
4,UK Office,1st Floor,9c:8c:6e:46:0b:7c,51.460782,-0.932292,2020-09-01 23:58:29.827
...,...,...,...,...,...,...
3733370,UK Office,1st Floor,c4:6e:1f:1a:68:a1,51.460701,-0.932252,2020-03-21 23:26:33.958
3733371,UK Office,1st Floor,a4:e9:75:77:7e:c3,51.460475,-0.932350,2020-03-21 23:26:28.204
3733372,UK Office,1st Floor,9c:8c:6e:46:0b:7c,51.460774,-0.932317,2020-03-21 23:26:27.926
3733373,UK Office,1st Floor,c4:6e:1f:1a:68:a1,51.460701,-0.932252,2020-03-21 23:26:27.853


In [8]:
from collections import defaultdict

# Create a dictionary of location exchanges keyed by (ClientMacAddr, lat, lng) tuple
location_exchanges = defaultdict(int)
MacFreq = defaultdict(int)

for _, row in datat.iterrows():
    MacFreq[row['ClientMacAddr']] += 1
    location_exchanges[(row['ClientMacAddr'], row['lat'], row['lng'])] += 1

# Create a dictionary of location counts keyed by ClientMacAddr
location_counts = defaultdict(int)
for key in location_exchanges:
    location_counts[key[0]] += 1

# Create a dictionary of ID statuses keyed by ClientMacAddr
ID_Status = defaultdict(str)
for key in location_counts:
    if location_counts[key] > 50:
        ID_Status[key] = 'Mobile'
    else:
        ID_Status[key] = 'Fixed'

EmployeeOrVisitor = defaultdict(str)
for key in MacFreq:
    if MacFreq[key] > 500:
        EmployeeOrVisitor[key] = 'Employee'
    else:
        EmployeeOrVisitor[key] = 'Visitor'

In [9]:
location_exchanges

defaultdict(int,
            {('9c:8c:6e:46:1c:5e',
              51.46044504990356,
              -0.9330483225094548): 25569,
             ('9c:8c:6e:46:0b:7c',
              51.46078771483341,
              -0.9322875162916772): 104,
             ('9c:8c:6e:46:0b:7c', 51.46078227015485, -0.9322924147881818): 88,
             ('9c:8c:6e:46:1c:5e',
              51.460447921188994,
              -0.9330476962056928): 1626,
             ('04:ea:56:92:f9:9b',
              51.460368222930526,
              -0.9325274505846268): 769,
             ('04:ea:56:92:f9:9b',
              51.46036830928204,
              -0.9325252800312868): 224,
             ('9c:8c:6e:46:0b:7c',
              51.460769537973015,
              -0.9323163796572012): 44,
             ('04:ea:56:92:f9:9b',
              51.460369125831605,
              -0.9325263131753722): 448,
             ('9c:8c:6e:46:0b:7c', 51.46077439313464, -0.932319248418118): 161,
             ('9c:8c:6e:46:1c:5e',
              51.46

In [11]:
location_counts

defaultdict(int,
            {'9c:8c:6e:46:1c:5e': 2741,
             '9c:8c:6e:46:0b:7c': 11923,
             '04:ea:56:92:f9:9b': 3132,
             'c0:b6:58:a2:3c:78': 3635,
             '00:80:92:df:7b:97': 309,
             '48:a4:72:ae:11:81': 1239,
             'e4:a7:a0:99:dc:9d': 2677,
             'b4:9c:df:63:d2:20': 614,
             '1c:5c:f2:f3:7a:fd': 37,
             '80:be:05:f2:1a:78': 3,
             'c8:b1:cd:0c:49:c1': 611,
             '88:66:a5:56:80:a0': 137,
             '88:66:a5:10:be:cb': 33,
             '40:40:a7:4a:f9:af': 21,
             '9c:da:3e:69:ca:8f': 189,
             '8c:8e:f2:9e:e8:eb': 5,
             '8c:b8:4a:b6:a4:46': 177,
             'cc:44:63:3c:40:8b': 115,
             '9c:da:3e:7e:f7:8e': 6,
             '88:66:a5:da:26:f7': 71,
             'a4:c3:f0:a5:f0:d2': 182,
             '00:b3:62:54:8c:ad': 6,
             '5c:f7:e6:e5:28:20': 162,
             '98:10:e8:06:8e:52': 192,
             'b4:9c:df:31:89:ea': 1133,
            

In [12]:
MacFreq

defaultdict(int,
            {'9c:8c:6e:46:1c:5e': 1407453,
             '9c:8c:6e:46:0b:7c': 1407403,
             '04:ea:56:92:f9:9b': 194188,
             'c0:b6:58:a2:3c:78': 8230,
             '00:80:92:df:7b:97': 5058,
             '48:a4:72:ae:11:81': 7928,
             'e4:a7:a0:99:dc:9d': 19553,
             'b4:9c:df:63:d2:20': 2580,
             '1c:5c:f2:f3:7a:fd': 87,
             '80:be:05:f2:1a:78': 10,
             'c8:b1:cd:0c:49:c1': 2621,
             '88:66:a5:56:80:a0': 587,
             '88:66:a5:10:be:cb': 102,
             '40:40:a7:4a:f9:af': 39,
             '9c:da:3e:69:ca:8f': 919,
             '8c:8e:f2:9e:e8:eb': 89,
             '8c:b8:4a:b6:a4:46': 429,
             'cc:44:63:3c:40:8b': 371,
             '9c:da:3e:7e:f7:8e': 16,
             '88:66:a5:da:26:f7': 737,
             'a4:c3:f0:a5:f0:d2': 1043,
             '00:b3:62:54:8c:ad': 51,
             '5c:f7:e6:e5:28:20': 427,
             '98:10:e8:06:8e:52': 696,
             'b4:9c:df:31:89:ea': 

In [14]:
testdata = datat.loc[datat['ClientMacAddr'] == '80:be:05:f2:1a:78']
testdata

,Site,Level,ClientMacAddr,lat,lng,localtime
4440,UK Office,1st Floor,80:be:05:f2:1a:78,51.460465,-0.933149,2020-09-01 18:27:56.808
4442,UK Office,1st Floor,80:be:05:f2:1a:78,51.460463,-0.933142,2020-09-01 18:27:46.743
4444,UK Office,1st Floor,80:be:05:f2:1a:78,51.460463,-0.933142,2020-09-01 18:27:38.8
4449,UK Office,1st Floor,80:be:05:f2:1a:78,51.460463,-0.933142,2020-09-01 18:27:25.285
4452,UK Office,1st Floor,80:be:05:f2:1a:78,51.460463,-0.933142,2020-09-01 18:27:15.207
4455,UK Office,1st Floor,80:be:05:f2:1a:78,51.460463,-0.933142,2020-09-01 18:27:05.119
4456,UK Office,1st Floor,80:be:05:f2:1a:78,51.460463,-0.933142,2020-09-01 18:26:58.596
4462,UK Office,1st Floor,80:be:05:f2:1a:78,51.460463,-0.933142,2020-09-01 18:26:45.049
4465,UK Office,1st Floor,80:be:05:f2:1a:78,51.460463,-0.933142,2020-09-01 18:26:34.989
4468,UK Office,1st Floor,80:be:05:f2:1a:78,51.460466,-0.933148,2020-09-01 18:26:24.97


In [9]:
res = []
for i, s in zip(ID_Status.keys(), ID_Status.values()):
  res.append([i, s])

dm = pd.DataFrame(res, columns=['ClientMacAddr', 'Status'])
dm

,ClientMacAddr,Status
0,9c:8c:6e:46:1c:5e,Mobile
1,9c:8c:6e:46:0b:7c,Mobile
2,04:ea:56:92:f9:9b,Mobile
3,c0:b6:58:a2:3c:78,Mobile
4,00:80:92:df:7b:97,Mobile
...,...,...
1388,a4:6c:f1:41:f3:9c,Fixed
1389,ec:1f:72:28:28:9d,Fixed
1390,34:8a:7b:81:9e:e6,Fixed
1391,48:d2:24:6e:23:5d,Fixed


In [10]:
ev = []
for i, s in zip(EmployeeOrVisitor.keys(), EmployeeOrVisitor.values()):
  ev.append([i, s])

dev = pd.DataFrame(ev, columns=['ClientMacAddr', 'EmployeeOrVisitor'])
dev

,ClientMacAddr,EmployeeOrVisitor
0,9c:8c:6e:46:1c:5e,Employee
1,9c:8c:6e:46:0b:7c,Employee
2,04:ea:56:92:f9:9b,Employee
3,c0:b6:58:a2:3c:78,Employee
4,00:80:92:df:7b:97,Employee
...,...,...
1388,a4:6c:f1:41:f3:9c,Visitor
1389,ec:1f:72:28:28:9d,Visitor
1390,34:8a:7b:81:9e:e6,Visitor
1391,48:d2:24:6e:23:5d,Visitor


In [11]:
dm['Status'].value_counts()

Fixed     1154
Mobile     239
Name: Status, dtype: int64

In [12]:
dev['EmployeeOrVisitor'].value_counts()

Visitor     1203
Employee     190
Name: EmployeeOrVisitor, dtype: int64

In [13]:
dm.to_csv(r'C:\Users\Jerry\OneDrive - University of Southern California\Desktop\ISE_534\000000000000_FixedorNot.csv', index=False)
dev.to_csv(r'C:\Users\Jerry\OneDrive - University of Southern California\Desktop\ISE_534\000000000000_EmployeeOrVisitor.csv', index=False)

In [ ]:
#test commit
#test 2